### Imports

In [22]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf
from finta import TA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

### Declare Constants

In [2]:
# The trading algorithm will be tested and evaluated over three timelines:

# dcb = Dot Com Bubble
dcb_start = '1997-06-01'
dcb_end = '2002-12-01'

# crsh = 2008 Crash
crsh_start = '2007-06-01'
crsh_end = '2012-12-01'

# cvd = COVID-19
cvd_start = '2020-03-01'
cvd_end = '2022-06-01'


### Get Financial Data

In [3]:
# Get price data from Yahoo! Finance for S&P 500, NASDAQ 100, and RUSSELL 2000
ohlvc_df = yf.download(
    '^GSPC ^NDX ^RUT', 
    start=dcb_start,
    end=cvd_end
)

# rename columns from tickers to descriptive names
ohlvc_df.rename(
    columns={
        '^NDX': 'NASDAQ 100',
        '^RUT': 'RUSSELL 2000',
        '^GSPC': 'SP 500', 
    },
    inplace=True
)

ohlvc_df.drop('Adj Close', axis='columns', level=0, inplace=True)
ohlvc_df.dropna(inplace=True)
ohlvc_df.columns = ohlvc_df.columns.swaplevel(0, 1)
ohlvc_df = ohlvc_df.sort_index(axis='columns')

[*********************100%***********************]  3 of 3 completed


### Prep `ohlvc_df`

In [4]:
stocks = list(dict.fromkeys(ohlvc_df.columns.get_level_values(0)))

for stock in stocks:

    ohlvc_df[stock, 'Actual Returns'] = ohlvc_df[stock, 'Close'].pct_change()

ohlvc_df = ohlvc_df.sort_index(axis='columns')

# Preview the data
ohlvc_df.head()

NASDAQ 100                                                  \
           Actual Returns       Close        High         Low        Open   
Date                                                                        
1997-06-02            NaN  958.690002  967.900024  950.950012  958.849976   
1997-06-03      -0.030124  929.809998  958.690002  929.479980  958.690002   
1997-06-04      -0.008443  921.960022  941.989990  915.960022  929.809998   
1997-06-05       0.009382  930.609985  935.750000  921.960022  921.960022   
1997-06-06       0.015452  944.989990  946.270020  926.979980  930.609985   

                        RUSSELL 2000                                      \
               Volume Actual Returns       Close        High         Low   
Date                                                                       
1997-06-02  555220000            NaN  383.519989  383.540009  380.589996   
1997-06-03  586410000      -0.000626  383.279999  383.720001  382.089996   
1997-06-04  594220000      -0.001591  382.670013  383.350006  381.839996   
1997-06-05  560950000       0.006063  384.989990  385.000000  382.670013   
1997-06-06  586010000       0.005585  387.140015  387.220001  384.720001   

                                          SP 500                          \
                  Open     Volume Actual Returns       Close        High   
Date                                                                       
1997-06-02  380.760010  435950000            NaN  846.359985  851.340027   
1997-06-03  383.519989  527120000      -0.001040  845.479980  850.559998   
1997-06-04  383.279999  466690000      -0.006351  840.109985  845.549988   
1997-06-05  382.670013  452610000       0.003952  843.429993  848.890015   
1997-06-06  384.989990  488940000       0.017287  858.010010  859.239990   

                                               
                   Low        Open     Volume  
Date                                           
1997-06-02  844.609985  848.280029  435950000  
1997-06-03  841.510010  846.359985  527120000  
1997-06-04  838.820007  845.479980  466690000  
1997-06-05  840.109985  840.109985  452610000  
1997-06-06  843.359985  843.429993  488940000

### Plot Closing Prices

In [5]:
ohlvc_px = ohlvc_df.xs('Close', level=1, axis=1)

px.line(ohlvc_px, width=1000)

## Calculate DMAC Signals

In [66]:
short_window = 30
long_window = 100

dmac_df = ohlvc_df.drop(
    columns=['Open', 'Low', 'High', 'Volume'], level=1
)

for stock in stocks:

    # Generate the fast and slow simple moving averages
    dmac_df[stock, 'SMA Fast'] = (
        dmac_df[stock, 'Close'].rolling(window=short_window).mean()
    )
    dmac_df[stock, 'SMA Slow'] = (
        dmac_df[stock, 'Close'].rolling(window=long_window).mean()
    )

    dmac_df[stock, 'Signal'] = 0.0
    dmac_df[stock, 'Signal'][short_window:] = np.where(
        dmac_df[stock, 'SMA Fast'][short_window:] < 
        dmac_df[stock, 'SMA Slow'][short_window:], 1.0, 0.0
    )

    dmac_df[stock, 'Entry/Exit'] = dmac_df[stock, 'Signal'].diff()

    dmac_df[stock, 'Close'] = dmac_df[stock, 'Close']

dmac_df = dmac_df.sort_index(axis='columns')
dmac_df.head()

NASDAQ 100                                                  \
           Actual Returns       Close Entry/Exit SMA Fast SMA Slow Signal   
Date                                                                        
1997-06-02            NaN  958.690002        NaN      NaN      NaN    0.0   
1997-06-03      -0.030124  929.809998        0.0      NaN      NaN    0.0   
1997-06-04      -0.008443  921.960022        0.0      NaN      NaN    0.0   
1997-06-05       0.009382  930.609985        0.0      NaN      NaN    0.0   
1997-06-06       0.015452  944.989990        0.0      NaN      NaN    0.0   

             RUSSELL 2000                                                  \
           Actual Returns       Close Entry/Exit SMA Fast SMA Slow Signal   
Date                                                                        
1997-06-02            NaN  383.519989        NaN      NaN      NaN    0.0   
1997-06-03      -0.000626  383.279999        0.0      NaN      NaN    0.0   
1997-06-04      -0.001591  382.670013        0.0      NaN      NaN    0.0   
1997-06-05       0.006063  384.989990        0.0      NaN      NaN    0.0   
1997-06-06       0.005585  387.140015        0.0      NaN      NaN    0.0   

                   SP 500                                                  
           Actual Returns       Close Entry/Exit SMA Fast SMA Slow Signal  
Date                                                                       
1997-06-02            NaN  846.359985        NaN      NaN      NaN    0.0  
1997-06-03      -0.001040  845.479980        0.0      NaN      NaN    0.0  
1997-06-04      -0.006351  840.109985        0.0      NaN      NaN    0.0  
1997-06-05       0.003952  843.429993        0.0      NaN      NaN    0.0  
1997-06-06       0.017287  858.010010        0.0      NaN      NaN    0.0

## Create DMAC Plot

In [67]:
def create_trading_plot(df, lines=list):

    entry_markers = df[df['Entry/Exit'] == 1.0]['Close']
    entry_markers.rename('Buy', inplace=True)

    exit_markers = df[df['Entry/Exit'] == -1.0]['Close']
    exit_markers.rename('Sell', inplace=True)


    price_sma_fig = px.line(df[lines])

    entries_fig = px.scatter(entry_markers)
    entries_fig.update_traces(
        marker=dict(
            symbol='triangle-up',
            color='green',
            size=15,
            line=dict(
                    width=1,
                    color='black'
                ),
            ),
        selector=dict(mode='markers')
    )

    exits_fig = px.scatter(exit_markers)
    exits_fig.update_traces(
        marker=dict(
            symbol='triangle-down',
            color='red',
            size=15,
            line=dict(
                    width=1,
                    color='black'
                ),
            ),
        selector=dict(mode='markers')
    )

    all_fig = go.Figure(
        data=price_sma_fig.data + entries_fig.data + exits_fig.data
    )

    all_fig.update_layout(width=1500, height=800)

    return all_fig


In [68]:
dmac_sp500_go = create_trading_plot(
    dmac_df['SP 500'], ['Close', 'SMA Fast', 'SMA Slow']).show()


## DMAC Portfolio Data

In [9]:
ptf_dmac_df = dmac_df.drop(
    columns=['Entry/Exit', 'SMA Fast', 'SMA Slow'], level=1
)

initial_capital = float(100000)
share_size = 100

for stock in stocks:

    ptf_dmac_df[stock, 'Position'] = share_size * ptf_dmac_df[stock, 'Signal']

    ptf_dmac_df[stock, 'Entry/Exit Position'] = ptf_dmac_df[stock, 'Position'].diff()

    ptf_dmac_df[stock, 'Holdings'] = ptf_dmac_df[stock, 'Close'] * ptf_dmac_df[stock, 'Position']

    ptf_dmac_df[stock, 'Cash'] = (
        initial_capital - (ptf_dmac_df[stock, 'Close'] * ptf_dmac_df[stock, 'Entry/Exit Position']).cumsum()
    )

    ptf_dmac_df[stock, 'Total'] = ptf_dmac_df[stock, 'Cash'] + ptf_dmac_df[stock, 'Holdings']

    ptf_dmac_df[stock, 'Daily Returns'] = ptf_dmac_df[stock, 'Total'].pct_change()
    
    ptf_dmac_df[stock, 'Cumulative Returns'] = (
        1 + ptf_dmac_df[stock, 'Daily Returns']
    ).cumprod() - 1

ptf_dmac_df = ptf_dmac_df.sort_index(axis='columns')
ptf_dmac_df

NASDAQ 100                                                 \
           Actual Returns          Cash         Close Cumulative Returns   
Date                                                                       
1997-06-02            NaN           NaN    958.690002                NaN   
1997-06-03      -0.030124  1.000000e+05    929.809998                NaN   
1997-06-04      -0.008443  1.000000e+05    921.960022           0.000000   
1997-06-05       0.009382  1.000000e+05    930.609985           0.000000   
1997-06-06       0.015452  1.000000e+05    944.989990           0.000000   
...                   ...           ...           ...                ...   
2022-05-24      -0.021974 -1.038781e+06  11769.839844           0.382028   
2022-05-25       0.014791 -1.038781e+06  11943.929688           0.556118   
2022-05-26       0.027869 -1.038781e+06  12276.790039           0.888978   
2022-05-27       0.032959 -1.038781e+06  12681.419922           1.293608   
2022-05-31      -0.003101 -1.038781e+06  12642.099609           1.254288   

                                                                            \
           Daily Returns Entry/Exit Position      Holdings Position Signal   
Date                                                                         
1997-06-02           NaN                 NaN  0.000000e+00      0.0    0.0   
1997-06-03           NaN                 0.0  0.000000e+00      0.0    0.0   
1997-06-04      0.000000                 0.0  0.000000e+00      0.0    0.0   
1997-06-05      0.000000                 0.0  0.000000e+00      0.0    0.0   
1997-06-06      0.000000                 0.0  0.000000e+00      0.0    0.0   
...                  ...                 ...           ...      ...    ...   
2022-05-24     -0.160611                 0.0  1.176984e+06    100.0    1.0   
2022-05-25      0.125967                 0.0  1.194393e+06    100.0    1.0   
2022-05-26      0.213904                 0.0  1.227679e+06    100.0    1.0   
2022-05-27      0.214206                 0.0  1.268142e+06    100.0    1.0   
2022-05-31     -0.017143                 0.0  1.264210e+06    100.0    1.0   

                           ...         SP 500                              \
                    Total  ... Actual Returns           Cash        Close   
Date                       ...                                              
1997-06-02            NaN  ...            NaN            NaN   846.359985   
1997-06-03  100000.000000  ...      -0.001040  100000.000000   845.479980   
1997-06-04  100000.000000  ...      -0.006351  100000.000000   840.109985   
1997-06-05  100000.000000  ...       0.003952  100000.000000   843.429993   
1997-06-06  100000.000000  ...       0.017287  100000.000000   858.010010   
...                   ...  ...            ...            ...          ...   
2022-05-24  138202.795410  ...      -0.008121 -155727.978516  3941.479980   
2022-05-25  155611.779785  ...       0.009451 -155727.978516  3978.729980   
2022-05-26  188897.814941  ...       0.019883 -155727.978516  4057.840088   
2022-05-27  229360.803223  ...       0.024742 -155727.978516  4158.240234   
2022-05-31  225428.771973  ...      -0.006274 -155727.978516  4132.149902   

                                                                 \
           Cumulative Returns Daily Returns Entry/Exit Position   
Date                                                              
1997-06-02                NaN           NaN                 NaN   
1997-06-03                NaN           NaN                 0.0   
1997-06-04            0.00000      0.000000                 0.0   
1997-06-05            0.00000      0.000000                 0.0   
1997-06-06            0.00000      0.000000                 0.0   
...                       ...           ...                 ...   
2022-05-24            1.38420     -0.013354                 0.0   
2022-05-25            1.42145      0.015624                 0.0   
2022-05-26            1.50056      0.0326

### Plot DMAC Cumulative Returns

In [74]:
ptf_dmac_px = ptf_dmac_df.xs('Cumulative Returns', level=1, axis=1).dropna()
px.line(ptf_dmac_px)

In [73]:
ptf_dmac_df['SP 500', 'Close']

Date
1997-06-02     846.359985
1997-06-03     845.479980
1997-06-04     840.109985
1997-06-05     843.429993
1997-06-06     858.010010
                 ...     
2022-05-24    3941.479980
2022-05-25    3978.729980
2022-05-26    4057.840088
2022-05-27    4158.240234
2022-05-31    4132.149902
Name: (SP 500, Close), Length: 6292, dtype: float64

# SVM

In [11]:
# Split any DataFrame into 75/25 train/test split
def get_training_dates(df):

    training_start = df.index.min()

    split_point = int(df.shape[0] * 0.75)
    training_end = df.iloc[split_point].name

    return training_start, training_end

In [79]:
svm_df = dmac_df.drop(columns=['Entry/Exit', 'Signal'], level=1)

for stock in stocks:

    svm_df[stock, 'Signal'] = 0.0
    svm_df[stock, 'Signal'] = np.where(
        (svm_df[stock, 'Actual Returns'] >= 0), 1, -1
    )

svm_df = svm_df.shift().dropna().sort_index(axis='columns')

In [81]:
training_start, training_end = get_training_dates(svm_df)

c_reports = []

for stock in stocks:

    X = pd.DataFrame()
    X['SMA Fast'] = svm_df[stock, 'SMA Fast']
    X['SMA Slow'] = svm_df[stock, 'SMA Slow']

    y = svm_df[stock, 'Signal'].copy()

    X_train = X.loc[training_start:training_end]
    y_train = y.loc[training_start:training_end]

    X_test = X.loc[training_end:]
    y_test = y.loc[training_end:]

    scaler = StandardScaler()

    scaler = scaler.fit(X_train)
    X_train_sc = scaler.transform(X_train)
    X_test_sc = scaler.transform(X_test)

    svm = SVC()

    svm = svm.fit(X_train_sc, y_train)

    y_pred = svm.predict(X_train_sc)

    c_reports.append(classification_report(y_train, y_pred, zero_division=1))

    print(X_train_sc)

[[-1.09979152 -1.15014858]
 [-1.09993504 -1.14869448]
 [-1.10096438 -1.14719032]
 ...
 [ 2.19705042  2.31176218]
 [ 2.20995493  2.31074497]
 [ 2.21925082  2.30906624]]
[[-0.95004603 -1.07934483]
 [-0.94710308 -1.07621789]
 [-0.94525185 -1.0734529 ]
 ...
 [ 1.63020104  1.70820958]
 [ 1.64352496  1.70490858]
 [ 1.65327185  1.7013402 ]]
[[-1.16559696 -1.25595061]
 [-1.16035313 -1.25199004]
 [-1.1563137  -1.24857817]
 ...
 [ 2.20305665  2.21458796]
 [ 2.21847114  2.21409957]
 [ 2.23035723  2.21302649]]


In [82]:
print(c_reports[0])

              precision    recall  f1-score   support

        -1.0       1.00      0.00      0.00      2135
         1.0       0.54      1.00      0.70      2510

    accuracy                           0.54      4645
   macro avg       0.77      0.50      0.35      4645
weighted avg       0.75      0.54      0.38      4645



In [83]:
print(c_reports[1])


              precision    recall  f1-score   support

        -1.0       0.57      0.01      0.02      2199
         1.0       0.53      0.99      0.69      2446

    accuracy                           0.53      4645
   macro avg       0.55      0.50      0.36      4645
weighted avg       0.55      0.53      0.37      4645



In [84]:
print(c_reports[2])

              precision    recall  f1-score   support

        -1.0       0.54      0.06      0.11      2177
         1.0       0.54      0.95      0.69      2468

    accuracy                           0.54      4645
   macro avg       0.54      0.51      0.40      4645
weighted avg       0.54      0.54      0.42      4645

